(GLM-poisson-regression)=
# GLM: Poisson Regression

:::{post} November 30, 2022
:tags: regression, poisson
:category: intermediate
:author: Jonathan Sedar, Benjamin Vincent
:::

This is a minimal reproducible example of Poisson regression to predict counts using dummy data.

This Notebook is basically an excuse to demo Poisson regression using PyMC, both manually and using `bambi` to demo interactions using the `formulae` library. We will create some dummy data, Poisson distributed according to a linear model, and try to recover the coefficients of that linear model through inference.

For more statistical detail see:

+ Basic info on [Wikipedia](https://en.wikipedia.org/wiki/Poisson_regression)
+ GLMs: Poisson regression, exposure, and overdispersion in Chapter 6.2 of [ARM, Gelmann & Hill 2006](http://www.stat.columbia.edu/%7Egelman/arm/)
+ This worked example from ARM 6.2 by [Clay Ford](http://www.clayford.net/statistics/poisson-regression-ch-6-of-gelman-and-hill/)

This very basic model is inspired by [a project by Ian Osvald](http://ianozsvald.com/2016/05/07/statistically-solving-sneezes-and-sniffles-a-work-in-progress-report-at-pydatalondon-2016/), which is concerned with understanding the various effects of external environmental factors upon the allergic sneezing of a test subject.

In [ ]:
#!pip install seaborn

In [ ]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import seaborn as sns

from formulae import design_matrices

In [ ]:
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)

%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

## Local Functions

## Generate Data

This dummy dataset is created to emulate some data created as part of a study into quantified self, and the real data is more complicated than this. Ask Ian Osvald if you'd like to know more [@ianozvald](https://twitter.com/ianozsvald).


### Assumptions:

+ The subject sneezes N times per day, recorded as `nsneeze (int)`
+ The subject may or may not drink alcohol during that day, recorded as `alcohol (boolean)`
+ The subject may or may not take an antihistamine medication during that day, recorded as the negative action `nomeds (boolean)`
+ We postulate (probably incorrectly) that sneezing occurs at some baseline rate, which increases if an antihistamine is not taken, and further increased after alcohol is consumed.
+ The data is aggregated per day, to yield a total count of sneezes on that day, with a boolean flag for alcohol and antihistamine usage, with the big assumption that nsneezes have a direct causal relationship.


Create 4000 days of data: daily counts of sneezes which are Poisson distributed w.r.t alcohol consumption and antihistamine usage

In [ ]:
# decide poisson theta values
theta_noalcohol_meds = 1  # no alcohol, took an antihist
theta_alcohol_meds = 3  # alcohol, took an antihist
theta_noalcohol_nomeds = 6  # no alcohol, no antihist
theta_alcohol_nomeds = 36  # alcohol, no antihist

# create samples
q = 1000
df = pd.DataFrame(
    {
        "nsneeze": np.concatenate(
            (
                rng.poisson(theta_noalcohol_meds, q),
                rng.poisson(theta_alcohol_meds, q),
                rng.poisson(theta_noalcohol_nomeds, q),
                rng.poisson(theta_alcohol_nomeds, q),
            )
        ),
        "alcohol": np.concatenate(
            (
                np.repeat(False, q),
                np.repeat(True, q),
                np.repeat(False, q),
                np.repeat(True, q),
            )
        ),
        "nomeds": np.concatenate(
            (
                np.repeat(False, q),
                np.repeat(False, q),
                np.repeat(True, q),
                np.repeat(True, q),
            )
        ),
    }
)

In [ ]:
df.tail()

##### View means of the various combinations (Poisson mean values)

In [ ]:
df.groupby(["alcohol", "nomeds"]).mean().unstack()

### Briefly Describe Dataset

In [ ]:
g = sns.catplot(
    x="nsneeze",
    row="nomeds",
    col="alcohol",
    data=df,
    kind="count",
    height=4,
    aspect=1.5,
)
for ax in (g.axes[1, 0], g.axes[1, 1]):
    for n, label in enumerate(ax.xaxis.get_ticklabels()):
        label.set_visible(n % 5 == 0)

**Observe:**

+ This looks a lot like poisson-distributed count data (because it is)
+ With `nomeds == False` and `alcohol == False` (top-left, akak antihistamines WERE used, alcohol was NOT drunk) the mean of the poisson distribution of sneeze counts is low.
+ Changing `alcohol == True` (top-right) increases the sneeze count `nsneeze` slightly
+ Changing `nomeds == True` (lower-left) increases the sneeze count `nsneeze` further
+ Changing both `alcohol == True and nomeds == True` (lower-right) increases the sneeze count `nsneeze` a lot, increasing both the mean and variance.

---

## Poisson Regression

Our model here is a very simple Poisson regression, allowing for interaction of terms:

$$ \theta = exp(\beta X)$$

$$ Y_{sneeze\_count} \sim Poisson(\theta)$$

**Create linear model for interaction of terms**

In [ ]:
fml = "nsneeze ~ alcohol + nomeds + alcohol:nomeds"  # full formulae formulation

In [ ]:
fml = "nsneeze ~ alcohol * nomeds"  # lazy, alternative formulae formulation

### 1. Manual method, create design matrices and manually specify model

**Create Design Matrices**

In [ ]:
dm = design_matrices(fml, df, na_action="error")

In [ ]:
mx_ex = dm.common.as_dataframe()
mx_en = dm.response.as_dataframe()

In [ ]:
mx_ex

**Create Model**

In [ ]:
with pm.Model() as mdl_fish:
    # define priors, weakly informative Normal
    b0 = pm.Normal("Intercept", mu=0, sigma=10)
    b1 = pm.Normal("alcohol", mu=0, sigma=10)
    b2 = pm.Normal("nomeds", mu=0, sigma=10)
    b3 = pm.Normal("alcohol:nomeds", mu=0, sigma=10)

    # define linear model and exp link function
    theta = (
        b0
        + b1 * mx_ex["alcohol"].values
        + b2 * mx_ex["nomeds"].values
        + b3 * mx_ex["alcohol:nomeds"].values
    )

    ## Define Poisson likelihood
    y = pm.Poisson("y", mu=pm.math.exp(theta), observed=mx_en["nsneeze"].values)

**Sample Model**

In [ ]:
with mdl_fish:
    inf_fish = pm.sample()
    # inf_fish.extend(pm.sample_posterior_predictive(inf_fish))

**View Diagnostics**

In [ ]:
az.plot_trace(inf_fish);

**Observe:**

+ The model converges quickly and traceplots looks pretty well mixed

### Transform coeffs and recover theta values

In [ ]:
az.summary(np.exp(inf_fish.posterior), kind="stats")

**Observe:**

+ The contributions from each feature as a multiplier of the baseline sneezecount appear to be as per the data generation:
    
    
    1. exp(Intercept): mean=1.05  cr=[0.98, 1.10]        
    
        Roughly linear baseline count when no alcohol and meds, as per the generated data: 

        theta_noalcohol_meds = 1 (as set above)
        theta_noalcohol_meds = exp(Intercept) 
                             = 1


    2. exp(alcohol): mean=2.86  cr=[2.67, 3.07]
    
        non-zero positive effect of adding alcohol, a ~3x multiplier of 
        baseline sneeze count, as per the generated data: 

        theta_alcohol_meds = 3 (as set above)
        theta_alcohol_meds = exp(Intercept + alcohol) 
                           = exp(Intercept) * exp(alcohol) 
                           = 1 * 3 = 3


    3. exp(nomeds): mean=5.73  cr=[5.34, 6.08]    
    
        larger, non-zero positive effect of adding nomeds, a ~6x multiplier of 
        baseline sneeze count, as per the generated data: 

        theta_noalcohol_nomeds = 6 (as set above)
        theta_noalcohol_nomeds = exp(Intercept + nomeds)
                               = exp(Intercept) * exp(nomeds) 
                               = 1 * 6 = 6
    
    
    4. exp(alcohol:nomeds): mean=2.10  cr=[1.96, 2.28]
    
        small, positive interaction effect of alcohol and meds, a ~2x multiplier of 
        baseline sneeze count, as per the generated data: 

        theta_alcohol_nomeds = 36 (as set above)
        theta_alcohol_nomeds = exp(Intercept + alcohol + nomeds + alcohol:nomeds)
                             = exp(Intercept) * exp(alcohol) * exp(nomeds * alcohol:nomeds)
                             = 1 * 3 * 6 * 2 = 36

### 2. Alternative method, using `bambi`

**Create Model**

**Alternative automatic formulation using `bambi`**

In [ ]:
model = bmb.Model(fml, df, family="poisson")

**Fit Model**

In [ ]:
inf_fish_alt = model.fit()

**View Traces**

In [ ]:
az.plot_trace(inf_fish_alt);

### Transform coeffs

In [ ]:
az.summary(np.exp(inf_fish_alt.posterior), kind="stats")

**Observe:**

+ The traceplots look well mixed
+ The transformed model coeffs look moreorless the same as those generated by the manual model
+ Note that the posterior predictive samples have an extreme skew

In [ ]:
posterior_predictive = model.predict(inf_fish_alt, kind="pps")

We can use `az.plot_ppc()` to check that the posterior predictive samples are similar to the observed data.

For more information on posterior predictive checks, we can refer to {ref}`pymc:posterior_predictive`.

In [ ]:
az.plot_ppc(inf_fish_alt);

## Authors
- Example originally contributed by [Jonathan Sedar](https://github.com/jonsedar) 2016-05-15.
- Updated to PyMC v4 by [Benjamin Vincent](https://github.com/drbenvincent) May 2022.
- Notebook header and footer updated November 2022.

## Watermark

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w -p pytensor,aeppl

:::{include} ../page_footer.md
:::